In [120]:
import requests
from lxml import etree
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
pd.set_option('max_columns',100)

In [182]:
html = 'http://fhy.wra.gov.tw/ReservoirPage_2011/Statistics.aspx?fbclid=IwAR3UAJHu0ILRyFcitXYliniQ3HsfvxTENs0iYS8x9m3BOoWWR5KKSkE3obk'
driver = webdriver.Chrome()

## 迴圈循環，年 - 月 - 日
for year in range(2014, 2020):
    for month in range(1, 13):
        for day in range(1, 32):
            driver.get(html)
            ## 查詢方式
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_cboSearch")]')
            elem.send_keys("全部")

            ## 年
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_ucDate_cboYear")]')
            elem.send_keys(year)
            ## 月
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_ucDate_cboMonth")]')
            elem.send_keys(month)
            ## 日
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_ucDate_cboDay")]')
            elem.send_keys(day)

            ## 時
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_ucDate_cboHour")]')
            elem.send_keys(10)
            ## 分
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_ucDate_cboMinute")]')
            elem.send_keys(0)

            ## 按下查詢
            elem = driver.find_element_by_xpath('//*[(@id = "ctl00_cphMain_btnQuery")]')
            elem.click()
            time.sleep(8)

            ## bs讀取
            page = BeautifulSoup(driver.page_source,'html5lib')
            ## 抓表格
            tables = page.select('table')
            ## 轉成表格
            df_list = []
            for table in tables:
                df_list.append(pd.concat(pd.read_html(table.prettify())))
            df = pd.concat(df_list)
            ## 篩選欄位
            df = df.loc[:, [('水庫名稱', '水庫名稱', '水庫名稱', '水庫名稱'),
             ('水情時間', '水情時間', '水情時間', '水情時間'),
             ('本日集水區  累積降雨量  (mm)',
              '本日集水區  累積降雨量  (mm)',
              '本日集水區  累積降雨量  (mm)',
              '本日集水區  累積降雨量  (mm)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '進流量  (cms)', '進流量  (cms)', '進流量  (cms)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水位  (公尺)', '水位  (公尺)', '水位  (公尺)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '滿水位  (公尺)', '滿水位  (公尺)', '滿水位  (公尺)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊',
              '有效蓄水量  (萬立方公尺)',
              '有效蓄水量  (萬立方公尺)',
              '有效蓄水量  (萬立方公尺)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '蓄水  百分比(%)', '蓄水  百分比(%)', '蓄水  百分比(%)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水庫出流量(cms)', '取水流量(cms)', '取水流量(cms)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水庫出流量(cms)', '發電  放水口', '目前狀態'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水庫出流量(cms)', '排砂道  /PRO', '預定時間'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水庫出流量(cms)', '排洪  隧道', '預定放流量  (cms)'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水庫出流量(cms)', '溢洪道', 'Unnamed: 12_level_3'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '水庫出流量(cms)', '小計', 'Unnamed: 13_level_3'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '防洪運轉狀態', '目前狀態', 'Unnamed: 14_level_3'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '防洪運轉狀態', '預定時間', 'Unnamed: 15_level_3'),
             ('颱風豪雨期間時，「防汛重點水庫」需登錄下列資訊', '防洪運轉狀態', '預定放流量  (cms)', 'Unnamed: 16_level_3')]]
            ## 改欄位名稱
            df.columns = (["水庫名稱", "水情時間", "本日集水區累積降雨量(mm)", "進流量(cms)"
                                        , "水位(公尺)", "滿水位(公尺)", "有效蓄水量(萬立方公尺)", "蓄水百分比(%)"
                                         , "取水流量(cms)", "發電放水口", "排砂道/PRO", "排洪隧道", "溢洪道"
                                        , "小計", "目前狀態", "預定時間", "預定放流量(cms)"])
            ## 存檔
            file = "C:\\Users\\admin\\Desktop\\水庫\\水庫\\" + str(year) + "-" + str(month) + "-" + str(day) + ".csv"
            df.to_csv(file, encoding='utf_8_sig')
            #df.to_csv("df.csv", encoding='utf_8_sig')

In [115]:
##### 讀不到 selenium 的網頁
res = requests.get(driver.page_source)
res_elements = etree.HTML(res.text)
## 抓表格
table = res_elements.xpath('//td')
## 跳出最後兩行沒用的
table.pop(-1)
table.pop(-1)
## 稍微清一下
newtable = []
for i in range(len(table)):
    a = etree.tostring(table[i], encoding='utf-8').decode()
    a = a.replace("</a>&#13;\n                            </td>'", "")
    a = a.replace("<td>'", "")
    a = a.replace("<td>", "")
    a = a.replace("</td>", "")
    a = a.replace('<td>&#13;\n                                --&#13;\n                            ', "")
    a = a.replace('&#13;\n                                --&#13;\n                            ', "")
    a = a.replace('&#13;\n                                --&#13;\n                            ', "")
    a = a.replace('&#13;\n                                --&#13;\n                            &#13;\n\t\t\t', "")
    a = a.replace('&#13;\n                                --&#13;\n                            ', "")
    a = a.replace('&#13;\n\t\t\t', "")
    a = a.replace('<td align="right">', "")
    a = a.replace("<td>&#13;\n                                --&#13;\n                            </td>", "")
    a = a.replace("<td>&#13;\n                                --&#13;\n                            </td>&#13;\n\t\t\t", "")
    a = a.replace("\n", "")
    a = a.replace("\t", "")
    a = a.replace("&#13;", "") 
    newtable.append(a)
## 變成data frame
df = pd.DataFrame(columns = ["水庫名稱", "水情時間", "本日集水區累積降雨量(mm)", "進流量(cms)"
                            , "水位(公尺)", "滿水位(公尺)", "有效蓄水量(萬立方公尺)", "蓄水百分比(%)"
                             , "取水流量(cms)", "發電放水口", "排砂道/PRO", "排洪隧道", "溢洪道"
                            , "小計", "目前狀態", "預定時間", "預定放流量(cms)"])
a = 0
for i in range(0, len(newtable), 17):
    df.loc[a] = newtable[i:i+17]
    a +=1
## 存data frame，記得改路徑
df.to_csv("df.csv", encoding='utf_8_sig')

InvalidSchema: No connection adapters were found for '<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><title>水庫統計表</title><link href="App_Themes/Reservoir/style.css" type="text/css" rel="stylesheet" /><link href="App_Themes/Reservoir/tools.css" type="text/css" rel="stylesheet" /></head>
<body>
    <form name="aspnetForm" method="post" action="./Statistics.aspx?fbclid=IwAR3UAJHu0ILRyFcitXYliniQ3HsfvxTENs0iYS8x9m3BOoWWR5KKSkE3obk" id="aspnetForm">
<div>
<input type="hidden" name="ctl00_ctl02_HiddenField" id="ctl00_ctl02_HiddenField" value=";;AjaxControlToolkit, Version=3.0.20820.16598, Culture=neutral, PublicKeyToken=28f01b0e84b6d53e:zh-TW:707835dd-fa4b-41d1-89e7-6df5d518ffb5:411fea1c:865923e8:77c58d20:91bd373d:14b56adc:596d588c:8e72a662:acd642d2:269a19ae" tabindex="0" />



</div>

<script type="text/javascript">
//&lt;![CDATA[
var theForm = document.forms['aspnetFor